In [ ]:
import os
import json
import joblib
import mlflow
import mlflow.sklearn
from datetime import datetime


In [ ]:
# -----------------------------
# Artifact locations
# -----------------------------
BASE_DIR = "./CREDITCARD/MODEL"
MODEL_PATH = os.path.join(BASE_DIR, "model.pkl")
METRICS_PATH = os.path.join(BASE_DIR, "metrics.json")

# -----------------------------
# MLflow configuration
# -----------------------------
MLFLOW_EXPERIMENT_NAME = "creditcard-fraud"
MLFLOW_MODEL_NAME = "creditcard-rf-model"

RUN_NAME = f"register-existing-{datetime.utcnow().strftime('%Y%m%d-%H%M%S')}"


In [ ]:
# Validate artifacts exist
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"model not found at {MODEL_PATH}")

if not os.path.exists(METRICS_PATH):
    raise FileNotFoundError(f"metrics not found at {METRICS_PATH}")

print("✅ Artifacts validated")


In [ ]:
# Load model & metrics (explicit disk load)
model = joblib.load(MODEL_PATH)

with open(METRICS_PATH, "r") as f:
    metrics = json.load(f)

print("✅ Loaded model & metrics")
print(metrics)


In [ ]:
# Set MLflow experiment (EXPERIMENT TRACKING)
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)


In [ ]:
# Start MLflow run (tracking + registry)
with mlflow.start_run(run_name=RUN_NAME) as run:

    # -----------------------------
    # Track metrics
    # -----------------------------
    mlflow.log_metrics(metrics)

    # -----------------------------
    # Track artifacts
    # -----------------------------
    mlflow.log_artifact(MODEL_PATH)
    mlflow.log_artifact(METRICS_PATH)

    # -----------------------------
    # Register model
    # -----------------------------
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        registered_model_name=MLFLOW_MODEL_NAME
    )

    # -----------------------------
    # Useful tags
    # -----------------------------
    mlflow.set_tag("source", "post-training-registration")
    mlflow.set_tag("model_type", type(model).__name__)
    mlflow.set_tag("artifact_format", "joblib")

    run_id = run.info.run_id

print(f"✅ Experiment run logged (run_id={run_id})")
print("✅ Model registered to MLflow Registry")


In [ ]:
#Verify experiment run
from mlflow.tracking import MlflowClient

client = MlflowClient()

run = client.get_run(run_id)

print("📌 Run info")
print("Run ID:", run.info.run_id)
print("Experiment ID:", run.info.experiment_id)
print("Metrics:", run.data.metrics)
print("Tags:", run.data.tags)


In [ ]:
# Verify model registry
registered_models = client.search_registered_models()

for model in registered_models:
    print(f"\n📦 Model: {model.name}")
    for v in model.latest_versions:
        print(
            f"   └── Version: {v.version}, "
            f"Stage: {v.current_stage}, "
            f"Run ID: {v.run_id}"
        )


In [ ]:
model_package_arn = model_package.model_package_arn

sm_client.add_tags(
    ResourceArn=model_package_arn,
    Tags=[
        {"Key": "role", "Value": "challenger"},
        {"Key": "status", "Value": "staging"},
        {"Key": "experiment", "Value": EXPERIMENT_NAME},
        {"Key": "trial", "Value": trial_name}
    ]
)
